In [2]:
import load

<Figure size 1000x600 with 0 Axes>

In [5]:
wesad = load.WESAD()

Finished building DataFrame


In [ ]:
df = wesad.separate_and_feature_extract(sample_n=70000)

In [ ]:
y = df['label']

,label,subject,std_ACC_0,mean_ACC_0,std_ACC_1,mean_ACC_1,std_ACC_2,mean_ACC_2,ECG_ULF,ECG_LF,ECG_HF,ECG_UHF,std_EMG,mean_EMG,std_EDA,mean_EDA,std_Resp,mean_Resp,std_Temp,mean_Temp
0,1,S10,0.021268,0.889405,0.063625,0.021046,0.201152,-0.041348,0.0,0.0,0.0,0.003626,0.009653,-0.002545,0.030105,0.787654,2.581328,0.030529,0.115981,34.291607
1,1,S10,0.021135,0.889580,0.063760,0.020579,0.200702,-0.039128,0.0,0.0,0.0,0.002576,0.009721,-0.002476,0.030146,0.787731,2.579613,0.037631,0.116559,34.290993
2,1,S10,0.021033,0.889690,0.063731,0.019927,0.200611,-0.037157,0.0,0.0,0.0,0.002208,0.009642,-0.002357,0.029960,0.787858,2.562645,0.045621,0.115626,34.292137
3,1,S10,0.021132,0.889684,0.063185,0.019906,0.200755,-0.037911,0.0,0.0,0.0,0.002804,0.009721,-0.002401,0.029928,0.787592,2.550647,0.072467,0.115722,34.292122
4,1,S10,0.021127,0.889671,0.063349,0.020196,0.200653,-0.038379,0.0,0.0,0.0,0.003534,0.009719,-0.002490,0.029872,0.787523,2.555122,0.054938,0.116350,34.291271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,2,S9,0.031953,0.830737,0.014515,-0.034487,0.080385,-0.411446,0.0,0.0,0.0,0.003643,0.013897,-0.003024,0.332063,3.364410,4.223909,0.050795,0.091995,34.569504
2696,2,S9,0.031897,0.830854,0.014449,-0.034525,0.079926,-0.411444,0.0,0.0,0.0,0.003213,0.013897,-0.003032,0.332722,3.363477,4.199047,0.060203,0.092407,34.569031
2697,2,S9,0.031772,0.830868,0.014466,-0.034499,0.079285,-0.411618,0.0,0.0,0.0,0.001997,0.013879,-0.002950,0.332822,3.362155,4.182932,0.069391,0.092159,34.568169
2698,2,S9,0.031856,0.830919,0.014318,-0.034449,0.079188,-0.411749,0.0,0.0,0.0,0.001567,0.013943,-0.003047,0.333764,3.362070,4.230389,0.092945,0.092468,34.568066


In [16]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import load

# 假設你已經載入並預處理 WESAD 數據
# X 為 (samples, timesteps, features)，這裡以 EDA 信號為例
# y 為壓力狀態類別


# 載入資料
wesad = load.WESAD()
df = wesad.separate_and_feature_extract(sample_n=70000)

y = df['label']
X = df.drop(['label','subject'],axis=1)
X = X.to_numpy()



Using ready-made DataFrame


In [17]:
X

array([[2.14744128e-02, 8.89088343e-01, 6.25388672e-02, ...,
        4.41580636e-02, 1.17837168e-01, 3.42903824e+01],
       [2.14858787e-02, 8.89284228e-01, 6.23953697e-02, ...,
        7.15630668e-02, 1.18007973e-01, 3.42898560e+01],
       [2.16803877e-02, 8.89348571e-01, 6.23235762e-02, ...,
        5.10332380e-02, 1.17726304e-01, 3.42899590e+01],
       ...,
       [3.21990892e-02, 8.31232543e-01, 1.44399885e-02, ...,
        2.18331473e-02, 9.19002369e-02, 3.45665703e+01],
       [3.21091413e-02, 8.30964771e-01, 1.46026699e-02, ...,
        4.43058559e-02, 9.21709687e-02, 3.45668449e+01],
       [3.19724222e-02, 8.30674199e-01, 1.45295818e-02, ...,
        2.40899222e-02, 9.26876441e-02, 3.45669136e+01]])

In [19]:
X.shape

(2700, 18)

In [27]:
def sum(a,b,c):
    return a+b+c


sum(*[1,2], 5)

8

In [26]:
X2 = X.reshape(*X.shape, 1).shape
X2

(2700, 18, 1)

In [22]:

# 分割訓練測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立 1D CNN 模型
def build_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(32, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(64, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(128, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# 初始化模型
input_shape = (X_train.shape[0], X_train.shape[1])
num_classes = len(np.unique(y))
model = build_cnn(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 20
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# 評估模型
loss, acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc * 100:.2f}%')


Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 18), dtype=float32). Expected shape (None, 2160, 18), but input has incompatible shape (None, 18)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 18), dtype=float32)
  • training=True
  • mask=None